In [31]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score, confusion_matrix

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
from sklearn.utils import resample

In [12]:
import gc

In [6]:
df = pd.read_parquet('/kaggle/input/res-11a/res_11.pq')
df

,is_zero_loans5,is_zero_loans530,is_zero_loans3060,is_zero_loans6090,is_zero_loans90,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,pclose_flag,fclose_flag,...,enc_loans_credit_type_4,enc_loans_credit_type_5,enc_loans_credit_type_6,enc_loans_credit_type_7,rn_max,credit_history_length_max,num_loan_months_max,frac_loan_months_max,has_loans,target
index,,,,,,,,,,,,,,,,,,,,,
0,0.900000,1.000000,1.000000,1.000000,1.000000,0.600000,0.900000,0.900000,0.100000,0.200000,...,0.700000,0.000000,0.0,0.000000,10,24.0,1,0.600000,0,0
1,0.857143,0.714286,0.857143,0.857143,0.785714,0.714286,0.857143,0.785714,0.071429,0.142857,...,0.571429,0.000000,0.0,0.000000,14,24.0,2,1.000000,0,0
2,1.000000,0.666667,0.666667,0.666667,1.000000,0.333333,1.000000,0.666667,0.666667,0.666667,...,0.333333,0.000000,0.0,0.000000,3,24.0,4,1.000000,0,0
3,1.000000,1.000000,1.000000,1.000000,1.000000,0.533333,0.933333,0.933333,0.333333,0.400000,...,0.600000,0.066667,0.0,0.000000,15,24.0,1,0.750000,0,0
4,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.0,0.000000,1,24.0,1,1.000000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,0.818182,0.454545,1.000000,1.000000,1.000000,0.727273,1.000000,0.636364,0.181818,0.454545,...,0.545455,0.000000,0.0,0.090909,11,24.0,5,0.571429,0,0
2999996,0.923077,1.000000,0.923077,1.000000,1.000000,0.692308,0.846154,0.846154,0.384615,0.307692,...,0.615385,0.000000,0.0,0.076923,13,24.0,2,0.750000,0,0
2999997,0.900000,0.700000,1.000000,1.000000,1.000000,0.600000,1.000000,0.900000,0.100000,0.100000,...,0.700000,0.000000,0.0,0.000000,10,24.0,6,0.750000,0,0


In [7]:
target = pd.read_csv('/kaggle/input/dlfintechbki/data_for_competition/train_target.csv').flag
target

0          0
1          0
2          0
3          0
4          0
          ..
2999995    0
2999996    0
2999997    0
2999998    0
2999999    0
Name: flag, Length: 3000000, dtype: int64

In [8]:
df.loc[:,'target'] = target
df.head()

,is_zero_loans5,is_zero_loans530,is_zero_loans3060,is_zero_loans6090,is_zero_loans90,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,pclose_flag,fclose_flag,...,enc_loans_credit_type_3,enc_loans_credit_type_4,enc_loans_credit_type_5,enc_loans_credit_type_6,enc_loans_credit_type_7,rn_max,credit_history_length_max,num_loan_months_max,frac_loan_months_max,target
index,,,,,,,,,,,,,,,,,,,,,
0,9,10,10,10,10,6,9,9,1,2,...,2,7,0,0,0,10,23.0,1,0.083333,0
1,12,10,12,12,11,10,12,11,1,2,...,3,8,0,0,0,14,24.0,2,0.285714,0
2,3,2,2,2,3,1,3,2,2,2,...,2,1,0,0,0,3,23.0,8,0.521739,0
3,15,15,15,15,15,8,14,14,5,6,...,4,9,1,0,0,15,24.0,0,0.000000,0
4,1,1,1,1,1,1,1,1,1,1,...,1,0,0,0,0,1,0.0,0,0.000000,0


In [8]:
df.loc[:,'target'] = target
df.head()

,is_zero_loans5,is_zero_loans530,is_zero_loans3060,is_zero_loans6090,is_zero_loans90,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,pclose_flag,fclose_flag,...,enc_loans_credit_type_4,enc_loans_credit_type_5,enc_loans_credit_type_6,enc_loans_credit_type_7,rn_max,credit_history_length_max,num_loan_months_max,frac_loan_months_max,has_loans,target
index,,,,,,,,,,,,,,,,,,,,,
0,0.900000,1.000000,1.000000,1.000000,1.000000,0.600000,0.900000,0.900000,0.100000,0.200000,...,0.700000,0.000000,0.0,0.0,10,24.0,1,0.60,0,0
1,0.857143,0.714286,0.857143,0.857143,0.785714,0.714286,0.857143,0.785714,0.071429,0.142857,...,0.571429,0.000000,0.0,0.0,14,24.0,2,1.00,0,0
2,1.000000,0.666667,0.666667,0.666667,1.000000,0.333333,1.000000,0.666667,0.666667,0.666667,...,0.333333,0.000000,0.0,0.0,3,24.0,4,1.00,0,0
3,1.000000,1.000000,1.000000,1.000000,1.000000,0.533333,0.933333,0.933333,0.333333,0.400000,...,0.600000,0.066667,0.0,0.0,15,24.0,1,0.75,0,0
4,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.0,0.0,1,24.0,1,1.00,0,0


In [9]:
train, test = train_test_split(df, stratify=df.target, test_size=0.2, random_state=44)

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [10]:
train.shape

(2400000, 386)

In [11]:
test.shape

(600000, 386)

## Downsampling

In [13]:
train_maj = train[train.target == 0] 
train_min = train[train.target == 1]
train_min.shape

(85154, 386)

In [14]:
train_downs = resample(train_maj, replace=False, n_samples=len(train_min), random_state=44)
train_downs

,is_zero_loans5,is_zero_loans530,is_zero_loans3060,is_zero_loans6090,is_zero_loans90,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,pclose_flag,fclose_flag,...,enc_loans_credit_type_4,enc_loans_credit_type_5,enc_loans_credit_type_6,enc_loans_credit_type_7,rn_max,credit_history_length_max,num_loan_months_max,frac_loan_months_max,has_loans,target
index,,,,,,,,,,,,,,,,,,,,,
644196,1.000000,1.000000,1.000000,1.000000,1.000000,0.800000,1.000000,1.000000,0.000000,0.200000,...,0.600000,0.000000,0.0,0.000,10,24.0,1,0.428571,0,0
2971099,0.875000,0.875000,0.750000,0.750000,0.750000,0.750000,0.875000,0.875000,0.000000,0.125000,...,0.625000,0.000000,0.0,0.125,8,24.0,2,0.625000,0,0
500525,0.500000,0.500000,1.000000,1.000000,1.000000,0.500000,1.000000,0.500000,0.000000,0.500000,...,0.500000,0.000000,0.0,0.000,2,24.0,1,0.500000,0,0
740875,1.000000,1.000000,1.000000,1.000000,1.000000,0.809524,0.904762,0.904762,0.000000,0.047619,...,0.809524,0.000000,0.0,0.000,21,24.0,1,0.750000,0,0
1852306,1.000000,1.000000,1.000000,1.000000,1.000000,0.800000,0.800000,0.800000,0.000000,0.800000,...,0.000000,0.000000,0.0,0.000,5,24.0,1,1.000000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555734,0.909091,0.909091,1.000000,1.000000,1.000000,0.727273,1.000000,0.909091,0.454545,0.363636,...,0.636364,0.000000,0.0,0.000,11,24.0,1,0.428571,0,0
429512,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.500000,...,0.500000,0.000000,0.0,0.000,2,24.0,1,0.250000,0,0
2730644,0.882353,1.000000,0.941176,0.941176,0.941176,0.764706,0.941176,0.941176,0.117647,0.176471,...,0.647059,0.000000,0.0,0.000,17,24.0,8,0.600000,0,0


In [15]:
train = pd.concat((train_downs, train_min), ignore_index=True).sample(frac=1.0)
train

,is_zero_loans5,is_zero_loans530,is_zero_loans3060,is_zero_loans6090,is_zero_loans90,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,pclose_flag,fclose_flag,...,enc_loans_credit_type_4,enc_loans_credit_type_5,enc_loans_credit_type_6,enc_loans_credit_type_7,rn_max,credit_history_length_max,num_loan_months_max,frac_loan_months_max,has_loans,target
119407,1.000000,1.000000,1.000000,1.000000,1.0,0.666667,1.000000,1.000000,0.666667,0.666667,...,0.000000,0.333333,0.0,0.000000,3,24.0,1,0.750000,0,1
105812,0.666667,0.666667,0.666667,1.000000,1.0,0.333333,1.000000,0.666667,0.333333,0.666667,...,0.333333,0.000000,0.0,0.000000,3,24.0,2,0.333333,0,1
144764,0.619048,0.714286,0.952381,1.000000,1.0,0.857143,1.000000,0.619048,0.190476,0.238095,...,0.619048,0.000000,0.0,0.000000,21,24.0,7,1.000000,0,1
68001,0.666667,1.000000,1.000000,1.000000,1.0,0.666667,1.000000,1.000000,0.000000,0.333333,...,0.333333,0.000000,0.0,0.000000,3,24.0,1,0.250000,0,0
50585,1.000000,0.750000,1.000000,1.000000,1.0,0.000000,1.000000,1.000000,0.000000,0.500000,...,0.500000,0.000000,0.0,0.000000,4,24.0,1,0.444444,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47827,0.818182,0.909091,0.909091,1.000000,1.0,0.727273,0.727273,0.636364,0.181818,0.272727,...,0.545455,0.000000,0.0,0.000000,11,24.0,3,0.750000,0,0
15235,0.947368,0.842105,0.947368,0.947368,1.0,0.631579,0.789474,0.789474,0.315789,0.157895,...,0.473684,0.000000,0.0,0.052632,19,24.0,2,1.000000,0,0
79471,0.916667,0.666667,1.000000,1.000000,1.0,0.666667,0.916667,0.583333,0.416667,0.250000,...,0.416667,0.000000,0.0,0.000000,12,24.0,2,0.300000,0,0
148911,1.000000,0.764706,1.000000,1.000000,1.0,0.823529,1.000000,0.823529,0.294118,0.176471,...,0.588235,0.000000,0.0,0.000000,17,24.0,1,0.750000,0,1


In [16]:
train.target.value_counts()

target
1    85154
0    85154
Name: count, dtype: int64

In [17]:
X_train, y_train = train.drop(columns='target'), train['target']

In [18]:
X_test, y_test = test.drop(columns='target'), test['target']

In [19]:
y_train

119407    1
105812    1
144764    1
68001     0
50585     0
         ..
47827     0
15235     0
79471     0
148911    1
45993     0
Name: target, Length: 170308, dtype: int64

In [20]:
del df
del train
del test
del train_downs
del train_maj
del train_min

In [21]:
gc.collect()

1516

### MLP torch

In [77]:
gc.collect()

0

In [78]:
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [79]:
torch.cuda.is_available()

True

In [80]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [81]:
class MyDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X.astype(np.float32))
        self.y = torch.from_numpy(y.astype(np.float32))
        self.X.to(torch.device("cuda:0"))
        self.y.to(torch.device("cuda:0"))
        
    def __getitem__(self, index):
        return self.X[index], self.y[index]
    
    def __len__(self):
        return self.X.shape[0]

# Инициализируем тренировочный torch.dataset
train_dataset = MyDataset(X_train.to_numpy(), y_train.to_numpy())

# Преобразуем тестовую выборку в torch-тензоры всю целиком
X_test_tensor = torch.from_numpy(X_test.to_numpy().astype(np.float32)).to(device)
y_test_tensor = torch.from_numpy(y_test.to_numpy().astype(np.float32)).to(device)
X_train_tensor = torch.from_numpy(X_train.to_numpy().astype(np.float32)).to(device)
y_train_tensor = torch.from_numpy(y_train.to_numpy().astype(np.float32)).to(device)

In [89]:
# Пусть размер батча у нас равен
batch_size = 2048

# Создайте Dataloader для тренировочной выборки на основе экземпляра train_dataset
train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
)

In [83]:
class ClassificationNet(nn.Module):
    def __init__(self, input_dim:int, hidden_dim:tuple, output_dim:int):
        super().__init__()
        
        self.hidden_list = []
        self.func_list = []
        
        prev_dim = input_dim
        for i, dim in enumerate(hidden_dim, 1):
            self.hidden_list.append(nn.Linear(prev_dim, dim))
            self.__setattr__(f"hidden{i}", self.hidden_list[-1])
            prev_dim = dim
            
            self.func_list.append(nn.ReLU())
            self.__setattr__(f"f{i}", self.func_list[-1])
        
        self.output = nn.Linear(prev_dim, output_dim)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
#         x.to(torch.device("cuda:0"))
        for hidden, func in zip(self.hidden_list, self.func_list):
            x = func(hidden(x))
#             x.to(torch.device("cuda:0"))
        
        x = self.sigmoid(self.output(x))
#         x.to(torch.device("cuda:0"))
        
        return x
    
    def predict(self, X):
        predict = [int(x > 0.5) for x in self(X)]
        return predict


# Объявляем экземпляр класса нейронной сети
net = ClassificationNet(X_train.shape[1], (2048, 1024, 512), 1).to(device)

net

ClassificationNet(
  (hidden1): Linear(in_features=385, out_features=2048, bias=True)
  (f1): ReLU()
  (hidden2): Linear(in_features=2048, out_features=1024, bias=True)
  (f2): ReLU()
  (hidden3): Linear(in_features=1024, out_features=512, bias=True)
  (f3): ReLU()
  (output): Linear(in_features=512, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [84]:
# Сюда будем сохранять значение roc_auc на тестовой выборке
# после каждой эпохи обучения
auc_test = []

In [96]:
# функция потерь
loss_fn = nn.BCELoss()

# Создаём оптимизатор
optimizer = torch.optim.SGD(net.parameters(), lr=0.0001)

# Делаем n эпох
num_epochs = 20

In [98]:
# Тренировочный цикл
for epoch in range(num_epochs):
    print(f"Epoch {epoch}: ", end='')
    for X, y in train_dataloader:
        X = X.to(device)
        y = y.to(device)
        pred = net(X).to(device)
        loss = loss_fn(pred, y.unsqueeze(-1))
        # print(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
  

    with torch.no_grad():
        # Конец эпохи: считаем функцию потерь на тестовой выборке, 
        # сохраняем в список, чтобы потом нарисовать график
        pred_proba = net(X_test_tensor).to(device)
        loss = loss_fn(
            pred_proba,
            y_test_tensor.unsqueeze(-1)
        ).item()
        auc = roc_auc_score(y_test, pred_proba.cpu())
        auc_test.append(auc)
        print(f'loss {loss}, auc {auc}')


Epoch 0: loss 0.588588535785675, auc 0.7554733588552353
Epoch 1: loss 0.5905629396438599, auc 0.7554654489934337
Epoch 2: loss 0.5899454355239868, auc 0.7554650605074585
Epoch 3: loss 0.5895625948905945, auc 0.7554693124645407
Epoch 4: loss 0.5898141860961914, auc 0.755470121945608
Epoch 5: loss 0.5907885432243347, auc 0.7554677963464788
Epoch 6: loss 0.5906506776809692, auc 0.7554613049942175
Epoch 7: loss 0.5897585153579712, auc 0.7554679101892661
Epoch 8: loss 0.5897324085235596, auc 0.7554691936297168
Epoch 9: loss 0.5909764766693115, auc 0.7554627513049375
Epoch 10: loss 0.5897232294082642, auc 0.7554703715068556
Epoch 11: loss 0.5903487801551819, auc 0.755476649906138
Epoch 12: loss 0.5897926688194275, auc 0.7554779319666762
Epoch 13: loss 0.5908774733543396, auc 0.7554745678209926
Epoch 14: loss 0.5900352001190186, auc 0.7554781326627844
Epoch 15: loss 0.5922084450721741, auc 0.7554634206031217
Epoch 16: loss 0.5906498432159424, auc 0.755470940152593
Epoch 17: loss 0.59047925472

Оценка качества предсказания:

In [99]:
with torch.no_grad():
    pred_net = net.predict(X_test_tensor)
    pred_net[:10]

In [100]:
confusion_matrix(y_test, pred_net)

array([[381157, 197555],
       [  5969,  15319]])

In [101]:
with torch.no_grad():
    print(roc_auc_score(y_test, net(X_test_tensor).cpu()))

0.7554673004302513


In [102]:
torch.save(net, 'nn_auc7554.pt')